## Exploratory Data Analysis

In [1]:
# --- Import necessary libraries ---
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar
import ipywidgets as widgets
from ipywidgets import interact

# --- Set global Matplotlib parameters ---
plt.rcParams["font.family"] = "serif"
plt.rcParams["figure.dpi"] = 1000
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["legend.fontsize"] = 20
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 5.0
plt.rcParams["xtick.minor.size"] = 3.0
plt.rcParams["ytick.major.size"] = 5.0
plt.rcParams["ytick.minor.size"] = 3.0
plt.rcParams["axes.linewidth"] = 1.5
plt.rcParams["legend.handlelength"] = 2.0

In [2]:
# --- Load training data (Years 1 and 2) ---
train_df = pd.read_excel('.././datasets/training.xlsx', sheet_name='Data')

# --- Load testing data (Year 3)
test_df = pd.read_excel('.././datasets/testing.xlsx', sheet_name='Data')

### Descriptive Statistics

In [5]:
# --- Dynamically find the columns for temperature and GHI ---
load_col = ['Load']
temp_cols = sorted([col for col in train_df.columns if 'Temp' in col])
ghi_cols = sorted([col for col in train_df.columns if 'GHI' in col])
all_cols = load_col + temp_cols + ghi_cols

# --- Create a list to hold the statistics for each year ---
stats_list = []

# --- Loop through each year in the training data ---
for year in sorted(train_df['Year'].unique()):
    # Filter data for the current year
    year_df = train_df[train_df['Year'] == year][all_cols]
    
    # Calculate descriptive statistics
    stats = {
        'Mean': year_df.mean(),
        'Variance': year_df.var(),
        'Median': year_df.median(),
        'Min': year_df.min(),
        'Max': year_df.max(),
        'Skewness': year_df.skew(),
        'Kurtosis': year_df.kurt()
    }
    
    # Convert dictionary to DataFrame and set index
    stats_df = pd.DataFrame(stats).T
    stats_df['Year'] = year
    stats_df = stats_df.set_index('Year', append=True).reorder_levels([1, 0])
    stats_list.append(stats_df)

# --- Concatenate the results for all years into a single DataFrame ---
summary_stats = pd.concat(stats_list)

# --- Create the multi-level column headers for the final table ---
summary_stats.columns = pd.MultiIndex.from_tuples([
    ('Load', ''),
    ('Temperature', 'Site 1'), ('Temperature', 'Site 2'), ('Temperature', 'Site 3'), 
    ('Temperature', 'Site 4'), ('Temperature', 'Site 5'),
    ('Global Horizontal Irradiance', 'Site 1'), ('Global Horizontal Irradiance', 'Site 2'), 
    ('Global Horizontal Irradiance', 'Site 3'), ('Global Horizontal Irradiance', 'Site 4'), 
    ('Global Horizontal Irradiance', 'Site 5')
])

print("Descriptive Statistics for Training Data (Years 1 & 2)")
display(summary_stats.style.format('{:.2f}'))

Descriptive Statistics for Training Data (Years 1 & 2)


### Interactive Plots

#### Hourly Variation Analysis
